<a href="https://colab.research.google.com/github/maticvl/dataHacker/blob/master/CNN/CNN_How_CNN_sees_the_world_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras import backend as K
import scipy
import cv2

Using TensorFlow backend.


In [0]:
model = InceptionV3(weights = 'imagenet')

In [0]:
model.output

<tf.Tensor 'predictions/Softmax:0' shape=(?, 1000) dtype=float32>

In [0]:
def preprocess(image_path):
    # load the image in the PIL format
    img = load_img(image_path, target_size = (299,299))
    # make a NumPy array of it
    img = img_to_array(img)
    # make a 4D tensor 
    img = np.expand_dims(img, axis = 0)
    # preprocessing steps for feeding the image through the inception_v3 
    img = preprocess_input(img)
    return img

In [0]:
def get_heatmap(model, img_path):
    image = preprocess(img_path)
    predictions = model.predict(image)
    data = decode_predictions(predictions, top = 3)[0]
    for i in range(3):
        print('The image was recognized as ',data[i][1],'with propability ',data[i][2])
    # the class which was recognized
    softmax_position = np.argmax(predictions)
    class_output = model.output[:,softmax_position]
    
    last_conv_layer = model.get_layer('conv2d_94')
    
    grads = K.gradients(class_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis = (0,1,2))
    
    # defining a function
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    
    pooled_grads_value, conv_layer_values = iterate([image])
    
    n_channels = grads.get_shape().as_list()[-1]
    
    for i in range(n_channels):
        conv_layer_values[:,:,i] *= pooled_grads_value[i]
    
    heatmap = np.mean(conv_layer_values, axis = -1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    
    return heatmap


In [0]:
def apply_heatmap(image_path, heatmap):
    img = cv2.imread(image_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    name = ''.join(list(image_path)[:-4])+'heatmap.jpg'
    cv2.imwrite(name, superimposed_img) 

In [0]:
heatmap1 = get_heatmap(model, 'cat1.jpg')
apply_heatmap('cat1.jpg', heatmap1)

The image was recognized as  Egyptian_cat with propability  0.8151559
The image was recognized as  Siamese_cat with propability  0.035323225
The image was recognized as  Angora with propability  0.014066448


In [0]:
heatmap1 = get_heatmap(model, 'cat2.jpg')
apply_heatmap('cat2.jpg', heatmap1)

The image was recognized as  Persian_cat with propability  0.9629857
The image was recognized as  Angora with propability  0.0011527061
The image was recognized as  Japanese_spaniel with propability  0.0010082541


In [0]:
heatmap1 = get_heatmap(model, 'english.jpg')
apply_heatmap('english.jpg', heatmap1)

The image was recognized as  English_foxhound with propability  0.81746376
The image was recognized as  Walker_hound with propability  0.07744286
The image was recognized as  German_short-haired_pointer with propability  0.004564064


In [0]:
heatmap1 = get_heatmap(model, 'retriver.jpg')
apply_heatmap('retriver.jpg', heatmap1)

The image was recognized as  golden_retriever with propability  0.84519035
The image was recognized as  Labrador_retriever with propability  0.0436717
The image was recognized as  Brittany_spaniel with propability  0.0041955807


In [0]:
heatmap1 = get_heatmap(model, 'jackrussel2.jpg')
apply_heatmap('jackrussel2.jpg', heatmap1)

The image was recognized as  toy_terrier with propability  0.42212814
The image was recognized as  English_foxhound with propability  0.17712013
The image was recognized as  wire-haired_fox_terrier with propability  0.07007175


In [0]:
heatmap1 = get_heatmap(model, 'labrador.jpg')
apply_heatmap('labrador.jpg', heatmap1)

The image was recognized as  Labrador_retriever with propability  0.91528267
The image was recognized as  golden_retriever with propability  0.046072792
The image was recognized as  Chesapeake_Bay_retriever with propability  0.0040103914
